## Analyze e2e latencies for the single_client_e2e experiment

In [ ]:
import numpy as np
import glob

def get_append_metrics(path):
    file_pattern = path + "append_metrics*.csv"

    total_throughput = 0
    latency_values = []

    for file in glob.glob(file_pattern):
        with open(file, 'r') as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(',')
                gsn, latency, throughput = int(parts[0]), float(parts[1]), float(parts[2])
                latency_values.append(latency)
            
            total_throughput += throughput

    latency_array = np.array(latency_values)

    mean_latency = np.mean(latency_array)
    p50_latency = np.percentile(latency_array, 50)
    p99_latency = np.percentile(latency_array, 99)

    print("results for computation time " + path.split("_")[-1].split("/")[0] + " us")
    print("statistic/metric, latency (us)")
    print(f"mean, {mean_latency:.2f}")
    print(f"p50, {p50_latency:.2f}")
    print(f"p99, {p99_latency:.2f}")
    print(f"total throughput, {total_throughput:.2f}")

    return int(path.split("_")[-1].split("/")[0]), mean_latency, total_throughput 

def get_e2e_metrics(path):
    file_pattern = path + "e2e_metrics*.csv"

    delivery_latency_values = []
    compute_latency_values = []
    confirmation_latency_values = []
    e2e_latency_values = []
    queuing_delay_values = []

    for file in glob.glob(file_pattern):
        with open(file, 'r') as f:
            lines = f.readlines()[1:]
            for line in lines:
                parts = line.strip().split(',')
                delivery, confirm, compute, e2e, queuing_delay = float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4]), float(parts[5])
                delivery_latency_values.append(delivery)
                compute_latency_values.append(compute)
                confirmation_latency_values.append(confirm)
                e2e_latency_values.append(e2e)
                queuing_delay_values.append(queuing_delay)

    
    delivery_latency_array = np.array(delivery_latency_values)
    compute_latency_array = np.array(compute_latency_values)
    confirmation_latency_array = np.array(confirmation_latency_values)
    e2e_latency_array = np.array(e2e_latency_values)
    queuing_delay_array = np.array([x for x in queuing_delay_values if x > 0])

    print("statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)")
    print(f"mean, {np.mean(delivery_latency_array):.2f}, {np.mean(compute_latency_array):.2f}, {np.mean(confirmation_latency_array):.2f}, {np.mean(e2e_latency_array):.2f}, {np.mean(queuing_delay_array):.2f}")
    print(f"std, {np.std(delivery_latency_array):.2f}, {np.std(compute_latency_array):.2f}, {np.std(confirmation_latency_array):.2f}, {np.std(e2e_latency_array):.2f}, {np.std(queuing_delay_array):.2f}")
    print(f"p50, {np.percentile(delivery_latency_array, 50):.2f}, {np.percentile(compute_latency_array, 50):.2f}, {np.percentile(confirmation_latency_array, 50):.2f}, {np.percentile(e2e_latency_array, 50):.2f}, {np.percentile(queuing_delay_array, 50):.2f}")
    print(f"p99, {np.percentile(delivery_latency_array, 99):.2f}, {np.percentile(compute_latency_array, 99):.2f}, {np.percentile(confirmation_latency_array, 99):.2f}, {np.percentile(e2e_latency_array, 99):.2f}, {np.percentile(queuing_delay_array, 99):.2f}")

    return int(path.split("_")[-1].split("/")[0]), np.mean(delivery_latency_array), np.mean(compute_latency_array), np.mean(confirmation_latency_array), np.mean(e2e_latency_array), np.mean(queuing_delay_array)

In [ ]:
path = "../results/*"

for dir in glob.glob(path + "*/"):
    get_append_metrics(dir)
    get_e2e_metrics(dir)


In [ ]:
import re
import pandas as pd

# Input data
data = """results for computation time 100 us
statistic/metric, latency (us)
mean, 3660.98
p50, 3540.00
p99, 8913.00
total throughput, 19439.58
results for computation time 100 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1199.89, 1355.14, 3686.58, 3686.63, 60.00
std, 874.39, 878.30, 1346.16, 1346.35, 39.96
p50, 1055.00, 1214.00, 3554.00, 3554.00, 68.00
p99, 4992.00, 5159.00, 9004.00, 9004.00, 110.00
results for computation time 200 us
statistic/metric, latency (us)
mean, 3612.39
p50, 3422.00
p99, 8900.00
total throughput, 19419.53
results for computation time 200 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1188.11, 1496.62, 3652.11, 3652.16, 111.81
std, 870.67, 877.52, 1386.00, 1386.31, 69.96
p50, 1037.00, 1349.00, 3449.00, 3449.00, 114.00
p99, 4940.00, 5261.00, 8964.00, 8964.00, 206.00
results for computation time 800 us
statistic/metric, latency (us)
mean, 3460.30
p50, 3142.00
p99, 8758.00
total throughput, 19464.46
results for computation time 800 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1222.99, 2435.65, 3500.47, 3512.67, 407.95
std, 918.98, 959.71, 1386.05, 1387.71, 239.83
p50, 1066.00, 2296.00, 3196.00, 3201.00, 408.00
p99, 5037.00, 6355.00, 8884.00, 8925.00, 800.00
results for computation time 1500 us
statistic/metric, latency (us)
mean, 3501.18
p50, 3237.00
p99, 9033.00
total throughput, 19411.71
results for computation time 1500 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1206.13, 3472.89, 3530.65, 3861.65, 760.11
std, 972.15, 1076.29, 1467.57, 1407.05, 437.99
p50, 1036.00, 3345.00, 3257.00, 3662.00, 761.00
p99, 5282.00, 7676.96, 9231.00, 9553.00, 1498.00
results for computation time 2500 us
statistic/metric, latency (us)
mean, 3707.25
p50, 3572.00
p99, 9063.00
total throughput, 19366.84
results for computation time 2500 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1210.32, 4987.93, 3740.00, 5081.85, 1265.78
std, 924.03, 1200.79, 1397.77, 1384.75, 736.89
p50, 1052.00, 4895.00, 3585.00, 4942.00, 1265.00
p99, 5088.00, 9141.00, 9146.00, 10115.00, 2496.00
results for computation time 1000 us
statistic/metric, latency (us)
mean, 3523.58
p50, 3215.00
p99, 8964.53
total throughput, 19408.73
results for computation time 1000 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1224.23, 2743.34, 3558.52, 3609.28, 511.16
std, 959.12, 1061.48, 1429.84, 1449.46, 341.79
p50, 1053.00, 2592.00, 3245.00, 3280.00, 509.00
p99, 5236.00, 6927.00, 9074.53, 9215.00, 1000.00
results for computation time 3000 us
statistic/metric, latency (us)
mean, 3448.60
p50, 3109.00
p99, 8949.00
total throughput, 19391.10
results for computation time 3000 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1210.46, 5741.43, 3491.49, 5789.11, 1517.19
std, 1065.57, 1401.64, 1608.88, 1644.23, 882.94
p50, 1031.00, 5642.00, 3157.00, 5657.00, 1516.00
p99, 5381.00, 10261.00, 9234.00, 10970.00, 2995.00
results for computation time 2000 us
statistic/metric, latency (us)
mean, 3629.14
p50, 3483.00
p99, 8503.00
total throughput, 19489.50
results for computation time 2000 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1190.24, 4211.09, 3659.39, 4391.92, 1011.95
std, 814.99, 1009.46, 1243.37, 1195.22, 583.33
p50, 1049.00, 4130.00, 3502.00, 4239.00, 1013.00
p99, 4745.00, 7949.00, 8638.61, 9080.00, 1997.00
results for computation time 2200 us
statistic/metric, latency (us)
mean, 3535.21
p50, 3287.00
p99, 9067.08
total throughput, 19380.78
results for computation time 2200 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1196.34, 4522.04, 3573.70, 4657.11, 1115.25
std, 959.28, 1171.25, 1462.58, 1399.58, 645.23
p50, 1020.00, 4411.00, 3319.00, 4493.00, 1116.00
p99, 5267.00, 8842.00, 9223.00, 10041.00, 2197.00
results for computation time 3800 us
statistic/metric, latency (us)
mean, 3688.30
p50, 3552.00
p99, 8819.00
total throughput, 19423.71
results for computation time 3800 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1187.89, 6926.64, 3710.40, 6953.74, 1921.44
std, 898.22, 1474.25, 1403.71, 1569.86, 1114.05
p50, 1039.00, 6864.00, 3560.00, 6873.00, 1920.00
p99, 4888.00, 11087.00, 8908.00, 11497.00, 3792.00
results for computation time 2800 us
statistic/metric, latency (us)
mean, 3643.12
p50, 3484.00
p99, 8738.00
total throughput, 19449.83
results for computation time 2800 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1187.63, 5414.31, 3679.01, 5474.53, 1415.22
std, 870.41, 1202.37, 1364.76, 1374.98, 815.83
p50, 1044.00, 5341.00, 3504.00, 5364.00, 1416.00
p99, 4835.00, 9346.00, 8838.00, 10055.00, 2794.00
results for computation time 1200 us
statistic/metric, latency (us)
mean, 3706.12
p50, 3527.00
p99, 9176.00
total throughput, 19360.17
results for computation time 1200 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1215.45, 3030.62, 3733.56, 3811.60, 609.51
std, 937.66, 1011.90, 1416.48, 1394.55, 351.00
p50, 1054.00, 2899.00, 3533.00, 3576.00, 610.00
p99, 5172.00, 7117.00, 9355.80, 9482.00, 1199.00
results for computation time 3300 us
statistic/metric, latency (us)
mean, 3494.79
p50, 3155.00
p99, 8901.00
total throughput, 19403.13
results for computation time 3300 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1230.23, 6211.30, 3527.89, 6247.25, 1667.75
std, 938.79, 1351.12, 1441.64, 1488.68, 961.54
p50, 1057.00, 6134.00, 3197.00, 6144.00, 1669.00
p99, 5163.00, 10479.00, 8982.00, 10997.00, 3293.00
results for computation time 5000 us
statistic/metric, latency (us)
mean, 3719.64
p50, 3565.00
p99, 8996.00
total throughput, 19373.27
results for computation time 5000 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1234.48, 8775.65, 3749.76, 8790.69, 2521.75
std, 1044.02, 1792.37, 1450.90, 1838.72, 1455.71
p50, 1058.00, 8716.00, 3572.00, 8722.00, 2519.00
p99, 5247.00, 13410.10, 9159.00, 13630.00, 4987.00
results for computation time 500 us
statistic/metric, latency (us)
mean, 3650.08
p50, 3434.00
p99, 8772.00
total throughput, 19448.02
results for computation time 500 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1214.38, 1975.15, 3711.82, 3712.22, 258.70
std, 861.88, 880.78, 1372.31, 1373.58, 150.89
p50, 1068.00, 1841.00, 3489.00, 3489.00, 259.00
p99, 4890.00, 5690.39, 8861.00, 8867.00, 501.00
results for computation time 4000 us
statistic/metric, latency (us)
mean, 3667.96
p50, 3517.00
p99, 8857.00
total throughput, 19427.85
results for computation time 4000 us
statistic/metric, delivery latency (us), compute latency (us), confirm latency (us), e2e latency (us), queuing delay (us)
mean, 1246.68, 7280.49, 3709.63, 7302.10, 2018.04
std, 845.72, 1443.28, 1293.40, 1520.47, 1164.80
p50, 1097.00, 7230.00, 3544.00, 7237.00, 2020.00
p99, 5026.00, 11491.00, 8919.00, 11842.00, 3990.00
"""

# Split data into lines
lines = data.splitlines()

# Parsing logic
results = []
current_time = None
append_latency = None

for i, line in enumerate(lines):
    line = line.strip()
    # Detect computation time
    if line.startswith("results for computation time"):
        match = re.search(r"computation time (\d+) us", line)
        if match:
            current_time = int(match.group(1))
    # Extract append latency (first "latency (us)" mean value)
    elif "statistic/metric, latency (us)" in line:
        append_line = lines[i + 1]  # Look at the next line
        append_match = re.match(r"mean,\s*([\d.]+)", append_line)
        if append_match:
            append_latency = float(append_match.group(1))
    # Extract other latencies (mean values)
    elif line.startswith("mean,") and current_time is not None:
        parts = line.split(",")
        if len(parts) == 6:  # Ensure correct format
            delivery_latency, compute_latency, confirm_latency, e2e_latency, queueing_delay = map(float, parts[1:])
            results.append({
                "computation_time": current_time,
                "append_latency": append_latency,
                "delivery_latency": delivery_latency,
                "compute_latency": compute_latency,
                "confirm_latency": confirm_latency,
                "e2e_latency": e2e_latency,
                "queueing_delay": queueing_delay,
            })

# Create a DataFrame
df = pd.DataFrame(results)

df.to_csv("output.csv", index=False)

# Print the CSV content
print(df.to_csv(index=False))